© 2023 summer https://fastcampus.co.kr/data_online_llama

In [1]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [2]:
from transformers import LlamaForCausalLM
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('news_gpt2')
model = LlamaForCausalLM.from_pretrained('news_llama')

model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(50257, 512, padding_idx=0)
    (layers): ModuleList(
      (0-3): 4 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=512, out_features=512, bias=False)
          (k_proj): Linear(in_features=512, out_features=512, bias=False)
          (v_proj): Linear(in_features=512, out_features=512, bias=False)
          (o_proj): Linear(in_features=512, out_features=512, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=512, out_features=1376, bias=False)
          (up_proj): Linear(in_features=512, out_features=1376, bias=False)
          (down_proj): Linear(in_features=1376, out_features=512, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_he

In [3]:
model.eval()

prompt = """\
If there were an annual prize for the "World\'s Most Hopeful Economy," it would likely go the"""
inputs = tokenizer(prompt, return_tensors="pt")
inputs.to(device)

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=100)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'If there were an annual prize for the "World\'s Most Hopeful Economy," it would likely go the way of the "World\'s Most Wanted" -- a list of the world\'s top 10 most wanted fugitives. The list of the top 10 most wanted fugitives is a list of the most wanted fugitives, with the most wanted fugitives, according to the list of the most wanted fugitives. The list includes the most wanted fugitives, with the most wanted fugitives, with the most wanted fugitives, according to the list'

In [4]:
prompt = """It's official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in"""
inputs = tokenizer(prompt, return_tensors="pt")
inputs.to(device)

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=50)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

"It's official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria, but the president has said he will not use force against ISIS. The president's comments came as the U.S. military"

# load dataset

In [5]:
from datasets import load_dataset
data = 'GonzaloA/fake_news'
dataset = load_dataset(data)

dataset

Repo card metadata block was not found. Setting CardData to empty.


DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'title', 'text', 'label'],
        num_rows: 24353
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'title', 'text', 'label'],
        num_rows: 8117
    })
    test: Dataset({
        features: ['Unnamed: 0', 'title', 'text', 'label'],
        num_rows: 8117
    })
})

In [6]:
dataset['train'][0]

{'Unnamed: 0': 0,
 'title': ' ‘Maury’ Show Official Facebook Posts F*CKED UP Caption On Guest That Looks Like Ted Cruz (IMAGE)',
 'text': 'Maury is perhaps one of the trashiest shows on television today. It s right in line with the likes of the gutter trash that is Jerry Springer, and the fact that those shows are still on the air with the shit they air really is a sad testament to what Americans find to be entertaining. However, Maury really crossed the line with a Facebook post regarding one of their guest s appearance with a vile, disgusting caption on Tuesday evening.There was a young woman on there doing one of their episodes regarding the paternity of her child. However, on the page, the show posted an image of the woman, who happens to bear a striking resemblance to Senator and presidential candidate Ted Cruz. The caption from the Maury Show page read: The Lie Detector Test determined .that was a LIE!  Ted Cruz is just NOT that SEXY! As if that weren t horrible enough, the capti

In [7]:
dataset['train'].features

{'Unnamed: 0': Value(dtype='int64', id=None),
 'title': Value(dtype='string', id=None),
 'text': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None)}

In [8]:
int2label = {0: 'False', 1: 'True'}
label2int = {'False': 0, 'True': 1}

In [9]:
prompt = """\
Determine if the given article is fake. article: ‘Maury’ Show Official Facebook Posts F*CKED UP Caption On Guest That Looks Like Ted Cruz (IMAGE) answer:"""
inputs = tokenizer(prompt, return_tensors="pt")
inputs.to(device)

# Generate
generate_ids = model.generate(inputs.input_ids, max_length = 60)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'Determine if the given article is fake. article: ‘Maury’ Show Official Facebook Posts F*CKED UP Caption On Guest That Looks Like Ted Cruz (IMAGE) answer: “I’m not a racist, I’m not a racist, I’m'

In [10]:
from datasets import DatasetDict
import random

prompt_format1 = """Determine if the given article is fake. article: %s  answer: %s"""
prompt_format2 = """Is this article fake? article: %s answer: %s"""
prompt_format3 = """Return True if the given article is fake. article: %s answer: %s"""

prompts = [prompt_format1, prompt_format2, prompt_format3]
def gen_prompt_fake(element):
    prompt_format = prompts[random.randint(0, len(prompts)-1)]
    return DatasetDict({'input': prompt_format%(element['title'], int2label[element['label']])})


dataset['train'] = dataset['train'].map(gen_prompt_fake)
dataset['validation'] = dataset['validation'].map(gen_prompt_fake)
dataset['test'] = dataset['test'].map(gen_prompt_fake)

Map:   0%|          | 0/24353 [00:00<?, ? examples/s]

Map:   0%|          | 0/8117 [00:00<?, ? examples/s]

Map:   0%|          | 0/8117 [00:00<?, ? examples/s]

In [11]:
dataset['train'][0]

{'Unnamed: 0': 0,
 'title': ' ‘Maury’ Show Official Facebook Posts F*CKED UP Caption On Guest That Looks Like Ted Cruz (IMAGE)',
 'text': 'Maury is perhaps one of the trashiest shows on television today. It s right in line with the likes of the gutter trash that is Jerry Springer, and the fact that those shows are still on the air with the shit they air really is a sad testament to what Americans find to be entertaining. However, Maury really crossed the line with a Facebook post regarding one of their guest s appearance with a vile, disgusting caption on Tuesday evening.There was a young woman on there doing one of their episodes regarding the paternity of her child. However, on the page, the show posted an image of the woman, who happens to bear a striking resemblance to Senator and presidential candidate Ted Cruz. The caption from the Maury Show page read: The Lie Detector Test determined .that was a LIE!  Ted Cruz is just NOT that SEXY! As if that weren t horrible enough, the capti

In [12]:
dataset['train'].to_pandas()

,Unnamed: 0,title,text,label,input
0,0,‘Maury’ Show Official Facebook Posts F*CKED U...,Maury is perhaps one of the trashiest shows on...,0,Return True if the given article is fake. arti...
1,1,Trump’s Favorite News Channel Tries To Soothe...,"Yesterday, after the father of one of the UCLA...",0,Return True if the given article is fake. arti...
2,2,"Russia warns Iraq, Kurds not to destabilize Mi...",MOSCOW (Reuters) - Russia on Wednesday warned ...,1,Return True if the given article is fake. arti...
3,3,WATCH STEVE SCALISE Throw A Strike At The Nati...,"House Majority Whip Steve Scalise (R., La.) th...",0,Determine if the given article is fake. articl...
4,4,Trump Will HATE What Stephen Colbert Just Did...,It can be said that Late Show host Stephen Col...,0,Is this article fake? article: Trump Will HAT...
...,...,...,...,...,...
24348,24348,EU Parliament chief asks Poland to ensure MEPs...,WARSAW (Reuters) - The president of the Europe...,1,Is this article fake? article: EU Parliament c...
24349,24349,Chemical weapons watchdog found sarin used in ...,AMSTERDAM/UNITED NATIONS (Reuters) - An inquir...,1,Is this article fake? article: Chemical weapon...
24350,24350,"Melissa Harris-Perry Is DONE With MSNBC, Pens...","As you may or may not know at this point, MSNB...",0,Is this article fake? article: Melissa Harris...
24351,24351,Trump's pick for Navy secretary withdraws,WASHINGTON (Reuters) - U.S. President Donald T...,1,Determine if the given article is fake. articl...


In [13]:
def tokenize(element):
    tokenizer.pad_token = tokenizer.eos_token
    outputs = tokenizer(
        element['input'],
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=False,
        return_length=True,
        padding=True
    )
    input_batch = []
    for inputs, input_ids, labels in zip(element["input"], outputs["input_ids"], element['label']):
        input_batch.append(input_ids)
    return {"input_ids": input_batch}


context_length=128
tokenized_datasets = dataset.map(
    tokenize, batched=True, remove_columns=dataset['train'].column_names
)
tokenized_datasets

Map:   0%|          | 0/24353 [00:00<?, ? examples/s]

Map:   0%|          | 0/8117 [00:00<?, ? examples/s]

Map:   0%|          | 0/8117 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 24353
    })
    validation: Dataset({
        features: ['input_ids'],
        num_rows: 8117
    })
    test: Dataset({
        features: ['input_ids'],
        num_rows: 8117
    })
})

# train

In [14]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [15]:
out = data_collator([tokenized_datasets['train'][i] for i in range(5)])
for key in out:
    print(f"{key} shape: {out[key].shape}")

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


input_ids shape: torch.Size([5, 91])
attention_mask shape: torch.Size([5, 91])
labels shape: torch.Size([5, 91])


In [38]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="fake_detect_llama",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="steps",
    eval_steps=1_00,
    logging_steps=1_00,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    weight_decay=0.1,
    warmup_steps=1_00,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=1_00,
    fp16=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

In [39]:
trainer.train() 

Step,Training Loss,Validation Loss
100,0.348000,3.704400
200,0.596700,3.689918
300,0.713500,3.649371
400,0.713700,3.618085
500,0.672200,3.608093


ZeroDivisionError: float division by zero

# evaluate

In [18]:
dataset['test'][234]['input']

'Determine if the given article is fake. article: Boeing CEO says he assured Trump about Air Force One costs  answer: True'

In [19]:
prompt = """Return True if the given article is fake. article: Boeing CEO says he assured Trump about Air Force One costs answer:"""

inputs = tokenizer(prompt, return_tensors="pt")
inputs.to(device)

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=50)
output = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

print(output)

Return True if the given article is fake. article: Boeing CEO says he assured Trump about Air Force One costs answer: True if the given article is fake. answer: True if the given article is fake. answer: True if the given article


In [20]:
tokenizer = AutoTokenizer.from_pretrained("news_gpt2", padding_side='left')


prompt_format1 = """Determine if the given article is fake. article: %s  answer:"""
prompt_format2 = """Is this article fake? article: %s answer:"""
prompt_format3 = """Return True if the given article is fake. article: %s answer:"""

prompts = [prompt_format1, prompt_format2, prompt_format3]

def gen_valid_prompt(element):
    prompt_format = prompts[random.randint(0, len(prompts)-1)]
    return DatasetDict({'input': prompt_format%(element['title'])})


valid_dataset = dataset['test'].select(range(100)).map(gen_valid_prompt)
valid_dataset[0]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

{'Unnamed: 0': 0,
 'title': 'FORMER U.S. ATTORNEY: “It’s VERY Clear Intel Conspired to Frame Trump” (VIDEO)',
 'text': 'JOE DIGENOVA has been around D.C for decades and has seen it all. He probably didn t see his one coming. The incoming president  was set-up to be taken down. A soft coup is in the works and DiGenova has this to say about it:"It\'s very clear that they conspired to frame the incoming President of the United States."  Joe diGenova on allegations of anti-Trump bias at FBI and TheJusticeDept #Tucker https://t.co/qUNjAenzJc pic.twitter.com/VDlhb45Ghi  G. Ashley Hawkins (@g_ashleyhawkins) December 16, 2017DiGenova on Tucker Carlson tonight: Inside the FBI and Department of Justice under Obama was a brazen plot to do two things. To exonerate Hillary Clinton because of an animous for Donald Trump, and then if she lost to frame the incoming president for either a criminal act or impeachment. This is one of the most disgusting performances by the senior officials at the FBI and

In [21]:
valid_dataset = valid_dataset.map(
    tokenize, batched=True, remove_columns=['text', 'input', 'Unnamed: 0', 'title']
)
valid_dataset

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Dataset({
    features: ['label', 'input_ids'],
    num_rows: 100
})

In [22]:
from torch.utils.data import DataLoader

batch_size=4
val_ds = valid_dataset
val_ds.set_format(type='torch')
val_dl = DataLoader(val_ds, batch_size=batch_size)

In [23]:
import re
import torch
from tqdm import tqdm

def acc(pred,label):
    return torch.sum(torch.tensor(pred) == label.squeeze()).item()


In [24]:
model_orig = LlamaForCausalLM.from_pretrained('news_llama')
model_orig.to(device)
model_orig.eval()

val_losses = []
val_acc = 0

for step, batch in enumerate(tqdm(val_dl)):
    label = batch['label']
    input_id= batch['input_ids'].to(device)

    pred = model_orig.generate(input_id, max_length=128)
    decoded_pred = tokenizer.batch_decode(pred, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    decoded_pred = [re.findall("answer: (True|False)", x)[0] if re.findall("answer: (True|False)", x) else 'none' for x in decoded_pred]
    decoded_pred = [label2int[x] if x in label2int else -1 for x in decoded_pred]

    val_acc += acc(decoded_pred, label)
    

print("val acc: ", val_acc/len(val_dl.dataset))

100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [00:09<00:00,  2.57it/s]

val acc:  0.0


In [25]:
tokenizer.batch_decode(pred, skip_special_tokens=True, clean_up_tokenization_spaces=False), label

(["Determine if the given article is fake. article: WATCH DIRTY HARRY REID ON HIS LIE ABOUT ROMNEY’S TAXES: “HE DIDN’T WIN, DID HE?”  answer: 'HE DIDN'TY HARRY, NO ONE OF THEMENT LIFE'S HEART: 'HE DIDN'TY HARRY, NO ONE OF THEMENT LIFE'S HEART: 'HE DIDN",
  "Is this article fake? article: North Korea diplomat says take atmospheric nuclear test threat 'literally' answer: 'I'm not sure if it's true.' The U.S. official said the U.S. is 'very concerned' about the North's nuclear program, but that the U.S. is 'very concerned' about the North's nuclear program. 'We are concerned about the North",
  "Is this article fake? article: VIRAL VIDEO: German Youth Deliver Powerful Anti-Refugee Message To Political Leaders: “We are ready for the Reconquista!” answer: 'We are ready to go to the World Cup. We are ready to go to the World Cup. We are ready to go to the World Cup. We are ready to go to the World Cup. We are ready to go to the World Cup. We are ready to go to the World Cup",
  'Is this arti

In [26]:
model.eval()
val_losses = []
val_acc = 0

for step, batch in enumerate(tqdm(val_dl)):
    label = batch['label']
    input_id= batch['input_ids'].to(device)

    pred = model.generate(input_id, max_length=150)
    decoded_pred = tokenizer.batch_decode(pred, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    decoded_pred = [re.findall("answer: ([^ ]*)", x)[0] if re.findall("answer: ([^ ]*)", x) else 'none' for x in decoded_pred]
    decoded_pred = [label2int[x] if x in label2int else -1 for x in decoded_pred]

    val_acc += acc(decoded_pred, label)
    

print("val acc: ", val_acc/len(val_dl.dataset))

100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [00:17<00:00,  1.47it/s]

val acc:  0.91


In [27]:
tokenizer.batch_decode(pred, skip_special_tokens=True, clean_up_tokenization_spaces=False), label

(['Determine if the given article is fake. article: WATCH DIRTY HARRY REID ON HIS LIE ABOUT ROMNEY’S TAXES: “HE DIDN’T WIN, DID HE?”  answer: False News For Trump’s “A “A*B*B*B*B*B*B*B*B*B*B*B*B*B*B*B*B*B*B*B*B*B*B*B*B*B*B*B*B*B*B*B*B*B*B*B*B',
  "Is this article fake? article: North Korea diplomat says take atmospheric nuclear test threat 'literally' answer: True if the given article is fake. answer: True if the given article is fake. answer: True if the given article is fake. answer: True if the given article is fake. article: Trump says U.S. sanctions on North Korea are 'unjust a threat' answer: True if the given article is fake. answer: True if the given article is fake. China's Xi says U",
  'Is this article fake? article: VIRAL VIDEO: German Youth Deliver Powerful Anti-Refugee Message To Political Leaders: “We are ready for the Reconquista!” answer: Falsely Stupidly uneased “Refugees” [Video] answer: Falsely Stupid Reason She’s A “A*cking” [Video] answer: Falsely Falsely Stupid R

In [28]:
model.save_pretrained('fake_detect_llama')